In [1]:
import pandas as pd
import numpy as np
# import sklearn

In [6]:
df = pd.read_csv('consolidated_df_sampled.csv').iloc[:,1:]

In [5]:
set(df.columns)
df.columns

Index(['Unnamed: 0', 'Project ID', 'Donation ID', 'Donor ID',
       'Donation Included Optional Donation', 'Donation Amount',
       'Donor Cart Sequence', 'Donation Received Date', 'Resource Item Name',
       'Resource Quantity', 'Resource Unit Price', 'Resource Vendor Name',
       'Donor City', 'Donor State', 'Donor Is Teacher', 'Donor Zip',
       'School ID', 'Teacher ID', 'Teacher Project Posted Sequence',
       'Project Type', 'Project Title', 'Project Essay',
       'Project Short Description', 'Project Need Statement',
       'Project Subject Category Tree', 'Project Subject Subcategory Tree',
       'Project Grade Level Category', 'Project Resource Category',
       'Project Cost', 'Project Posted Date', 'Project Expiration Date',
       'Project Current Status', 'Project Fully Funded Date', 'Teacher Prefix',
       'Teacher First Project Posted Date', 'School Name', 'School Metro Type',
       'School Percentage Free Lunch', 'School State', 'School Zip',
       'School Ci

In [7]:
n_rows = 10000
df_sample = df.sample(n = n_rows, random_state = 123, axis = 0)

In [8]:
not_useful = ['Project ID', 'Donation ID','Donor ID',
          'Donor Cart Sequence','Resource Vendor Name','Resource Item Name', 
              'Teacher Project Posted Sequence','School ID', 'Teacher ID',
          'Project Title', 'Project Essay','Project Short Description',
          'Project Need Statement','School Name', 'Donor Zip', 'School Zip']

date_feat = ['Donation Received Date','Project Posted Date', 'Project Expiration Date',
        'Project Fully Funded Date','Teacher First Project Posted Date']

donor_feat = ['Donation ID', 'Donor ID',
       'Donation Included Optional Donation', 'Donation Amount',
       'Donor Cart Sequence', 'Donation Received Date', 
       'Donor City', 'Donor State', 'Donor Is Teacher', 'Donor Zip']

project_feat = ['Resource Item Name',
       'Resource Quantity', 'Resource Unit Price', 'Resource Vendor Name',
       'Project Type', 'Project Title', 'Project Essay',
       'Project Short Description', 'Project Need Statement',
       'Project Subject Category Tree', 'Project Subject Subcategory Tree',
       'Project Grade Level Category', 'Project Resource Category',
       'Project Cost', 'Project Posted Date', 'Project Expiration Date',
       'Project Current Status', 'Project Fully Funded Date', 'Teacher Prefix',
       'Teacher First Project Posted Date', 'School Name', 'School Metro Type',
       'School Percentage Free Lunch', 'School State', 'School Zip',
       'School City', 'School County', 'School District']

cat_feat = ['Donor City', 'Donor State', 'Donor Is Teacher', 'Donor Zip',
            'Project Type','Project Subject Category Tree', 'Donation Included Optional Donation',
            'Project Subject Subcategory Tree',
            'Project Grade Level Category', 'Project Resource Category',
            'Project Current Status','Teacher Prefix','School Metro Type',
            'School State', 'School Zip','School City','School County', 'School District']

In [9]:
df_sample = df_sample.drop(labels=not_useful,axis = 1)

In [10]:
cat_feat_new = []
for x in df_sample.columns.tolist(): 
    if x in cat_feat:
        cat_feat_new.append(x)

In [11]:
df_sample.dropna(axis=0, inplace=True)

In [12]:
checker1 = df_sample[df_sample['School State']=='Alaska'].index.tolist()
checker2 = df_sample[df_sample['School Metro Type']=='town'].index.tolist()

In [13]:
from sklearn.preprocessing import LabelEncoder

labels = {}
for cat_feat_new in cat_feat_new:
    le = LabelEncoder()
    le.fit(df_sample[cat_feat_new].value_counts().index.values)
    
    if df_sample[cat_feat_new].dtype == 'float64' or\
    df_sample[cat_feat_new].dtype == 'int':
        df_sample[cat_feat_new] = le.transform(df_sample[cat_feat_new])
    
    else:
        df_sample[cat_feat_new] = le.transform(df_sample[cat_feat_new].astype(str))
    
    labels[cat_feat_new] = list(le.classes_)

In [14]:
df_sample['Project Posted Date'] = pd.to_datetime(
    df_sample['Project Posted Date'], errors='coerce')

df_sample['Project Expiration Date'] = pd.to_datetime(
    df_sample['Project Expiration Date'], errors='coerce')

df_sample['Project Fully Funded Date'] = pd.to_datetime(
    df_sample['Project Fully Funded Date'], errors='coerce')

df_sample['Donation Received Date'] = pd.to_datetime(
    df_sample['Donation Received Date'], errors='coerce')

df_sample['Teacher First Project Posted Date'] = pd.to_datetime(
    df_sample['Teacher First Project Posted Date'], errors='coerce')

df_sample['delta-days-before-expiry'] = (df_sample['Project Expiration Date'] - df_sample['Project Posted Date']).dt.days
df_sample['delta-days-before-funded'] = (df_sample['Project Fully Funded Date'] - df_sample['Project Posted Date']).dt.days
df_sample['delta-days-before-donating'] = (df_sample['Donation Received Date'] - df_sample['Project Posted Date']).dt.days

In [15]:
X = df_sample[['Resource Quantity', 'Resource Unit Price','Project Type',
       'Project Subject Category Tree', 'Project Subject Subcategory Tree',
       'Project Grade Level Category', 'Project Resource Category',
       'Project Cost', 'Project Current Status', 'Teacher Prefix',
       'School Metro Type','School Percentage Free Lunch', 'School State', 'School City',
       'School County', 'School District']]

In [17]:
from sklearn.cluster import KMeans

In [18]:
n_clusters = 4
kmeans = KMeans(n_clusters = n_clusters).fit(X)
plt.figure(figsize=(10,6))
for i in range(len(kmeans.cluster_centers_)):
    plt.plot(kmeans.cluster_centers_[i][0],kmeans.cluster_centers_[i][1],'x', markersize=12, label=i);
    plt.xticks([], [])
    plt.yticks([], [])
    plt.legend(loc=4)
plt.title('KMeans Clustering (n_clusters = 4)');

NameError: name 'plt' is not defined